In [2]:
import pandas as pd
import requests

Set up a path variable to access the data

In [3]:
shark_url = "http://api.fish.wa.gov.au/webapi/v1/RawData"
csv_path = "data/sharks/shark_sightings_raw.csv"

Download the data from the API

In [ ]:
response = requests.get(shark_url).json()


Load the data into a Pandas DF

In [7]:
shark_df = pd.DataFrame(response)

NameError: name 'response' is not defined

Save the data into a CSV file

In [81]:
shark_df.to_csv("data/sharks/shark_sightings_raw.csv")

NameError: name 'shark_df' is not defined

In [83]:
sharks_df = pd.read_csv(csv_path)
sharks_df

,Unnamed: 0,RawDataId,ObjectId,Distance,DistanceUnit,InteractionValue,InteractionId,TownProximity,LocationValue,SightingNumbervalue,ReportDateTime,SightingDateTime,SightingSizeValue,SightingSpeciesValue,OwnerValue,LocationDetail,LocationX,LocationY
0,0,63739,3073,NaN,NaN,Detected,4,NaN,Meelup,NaN,NaN,2021-07-24T17:48:00,NaN,white,Fisheries Advise,NaN,115.093091,-33.57247
1,1,63738,3073,NaN,NaN,Detected,4,NaN,Meelup,NaN,NaN,2021-07-24T17:43:00,NaN,white,Fisheries Advise,NaN,115.093091,-33.57247
2,2,63737,3073,NaN,NaN,Detected,4,NaN,Meelup,NaN,NaN,2021-07-24T17:38:00,NaN,white,Fisheries Advise,NaN,115.093091,-33.57247
3,3,63736,WA12582TSE,50.0,m offshore,sighted,1,Perth Metro,"Jervoise Bay Beach, Perth Metro",NaN,NaN,2021-07-24T12:15:12,2.5m,unknown sp.,Public report,NaN,115.765000,-32.13930
4,4,63735,WA51495TSE,600.0,m offshore,sighted,1,Coral Bay,"Bateman Bay, near Coral Bay",NaN,NaN,2021-07-24T11:24:29,NaN,whale carcass,Fisheries advise,Lots of sharks around carcass,113.807000,-23.05480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17309,17309,54674,WA00004TSE,10.0,m,Sighted,1,Dunsborough,"Bunker Bay, near Dunsborough",1,NaN,2014-10-14T03:40:00,2m,white,Public report,NaN,115.041100,-33.54306
17310,17310,54675,WA00003TSE,NaN,NaN,Sighted,1,Yallingup,"Yallingup Beach, Yallingup",1,NaN,2014-10-14T02:08:00,NaN,unknown sp.,Public report,NaN,115.028170,-33.63339
17311,17311,54676,WA00002TSE,NaN,NaN,Sighted,1,Dunsborough,"Dunsborough Boat Ramp, Dunsborough",1,2014-10-13T04:00:00,2014-10-13T03:55:00,4.5m,white,Public report,Old Dunsborough,115.105300,-33.60111
17312,17312,46307,1035,NaN,NaN,Detected,4,NaN,Garden Island (north end),NaN,NaN,2014-10-13T02:28:00,NaN,tiger,Fisheries Advise,NaN,115.642830,-32.14411


# Data Cleaning

## Species

Here the species present in the data are reviewed. The are several cases of duplicate values that need to be combined, and species values that are not sharks need to be removed.

In [46]:
sharks_df['SightingSpeciesValue'].value_counts()

bronze whaler                 5590
unknown sp.                   4422
tiger                         3878
white                         2541
hammerhead                     317
whale carcass                  190
whaler                         119
bull                            91
mako                            49
sonar object                    45
other                           29
dusky                           11
grey nurse                      10
blacktip                         4
whale carcass                    4
blue                             4
detection event - possible       4
shortfin mako                    1
white                            1
Whale Carcass                    1
Name: SightingSpeciesValue, dtype: int64

Replace duplicate entry values with a single common value.

In [80]:
clean_sharks = sharks_df.copy()
clean_sharks['SightingSpeciesValue'] = clean_sharks['SightingSpeciesValue'].replace({
    'Whale Carcass':'whale carcass',
    'whale carcass ':'whale carcass',
    'whaler':'bronze whaler',
    'shortfin mako':'mako', 
    'white ': 'white'})

Drop rows where the species value is not a shark.

In [82]:
# create a list of species values to filter out
drop_species = ['whale carcass',
                'detection event - possible',
                'other',
                'sonar object'] 

# use the list to create a filter
species_filter = ~clean_sharks.SightingSpeciesValue.isin(drop_species) 

# apply the filter to the dataset
clean_sharks = clean_sharks[species_filter] 

# display the cleaned species values
clean_sharks['SightingSpeciesValue'].value_counts()

bronze whaler    5709
unknown sp.      4422
tiger            3878
white            2542
hammerhead        317
bull               91
mako               50
dusky              11
grey nurse         10
blue                4
blacktip            4
Name: SightingSpeciesValue, dtype: int64

## Report Owner

There are several cases of duplicate entrys that need to be consolidated

In [100]:
clean_sharks['OwnerValue'].value_counts()

Fisheries Advise           10469
Public report               3070
SLS Westpac Heli report     2422
SLS Lifesavers report        630
SMART drumline trial         300
UPDATE to prior report       105
DBCA officers report          33
DPAW report                   10
Clever Buoy trial              1
Name: OwnerValue, dtype: int64

In [96]:
clean_sharks['OwnerValue'] = clean_sharks['OwnerValue'].replace({
    'Fisheries advise': 'Fisheries Advise',
    'SLS Lifesavers report ': 'SLS Lifesavers report',
    'SLS Westpac Heli': 'SLS Westpac Heli report',
    'Public Report': 'Public report'
})

# display the cleaned owner values
clean_sharks['OwnerValue'].value_counts()

Fisheries Advise           10469
Public report               3070
SLS Westpac Heli report     2422
SLS Lifesavers report        630
SMART drumline trial         300
UPDATE to prior report       105
DBCA officers report          33
DPAW report                   10
Clever Buoy trial              1
Name: OwnerValue, dtype: int64

## Interaction Type

text here

In [104]:
clean_sharks['InteractionId'].value_counts()

4    10166
1     6435
3      283
2      121
6       34
5        2
Name: InteractionId, dtype: int64

In [105]:
clean_sharks

,Unnamed: 0,RawDataId,ObjectId,Distance,DistanceUnit,InteractionValue,InteractionId,TownProximity,LocationValue,SightingNumbervalue,ReportDateTime,SightingDateTime,SightingSizeValue,SightingSpeciesValue,OwnerValue,LocationDetail,LocationX,LocationY
0,0,63739,3073,NaN,NaN,Detected,4,NaN,Meelup,NaN,NaN,2021-07-24T17:48:00,NaN,white,Fisheries Advise,NaN,115.093091,-33.57247
1,1,63738,3073,NaN,NaN,Detected,4,NaN,Meelup,NaN,NaN,2021-07-24T17:43:00,NaN,white,Fisheries Advise,NaN,115.093091,-33.57247
2,2,63737,3073,NaN,NaN,Detected,4,NaN,Meelup,NaN,NaN,2021-07-24T17:38:00,NaN,white,Fisheries Advise,NaN,115.093091,-33.57247
3,3,63736,WA12582TSE,50.0,m offshore,sighted,1,Perth Metro,"Jervoise Bay Beach, Perth Metro",NaN,NaN,2021-07-24T12:15:12,2.5m,unknown sp.,Public report,NaN,115.765000,-32.13930
5,5,63734,2332,NaN,NaN,Detected,4,NaN,Leighton,NaN,NaN,2021-07-24T03:14:00,NaN,white,Fisheries Advise,NaN,115.743650,-32.02628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17309,17309,54674,WA00004TSE,10.0,m,Sighted,1,Dunsborough,"Bunker Bay, near Dunsborough",1,NaN,2014-10-14T03:40:00,2m,white,Public report,NaN,115.041100,-33.54306
17310,17310,54675,WA00003TSE,NaN,NaN,Sighted,1,Yallingup,"Yallingup Beach, Yallingup",1,NaN,2014-10-14T02:08:00,NaN,unknown sp.,Public report,NaN,115.028170,-33.63339
17311,17311,54676,WA00002TSE,NaN,NaN,Sighted,1,Dunsborough,"Dunsborough Boat Ramp, Dunsborough",1,2014-10-13T04:00:00,2014-10-13T03:55:00,4.5m,white,Public report,Old Dunsborough,115.105300,-33.60111
17312,17312,46307,1035,NaN,NaN,Detected,4,NaN,Garden Island (north end),NaN,NaN,2014-10-13T02:28:00,NaN,tiger,Fisheries Advise,NaN,115.642830,-32.14411


## Filtering

Select sightings between a specified date range

In [111]:
datefiltered_df = clean_sharks.copy().loc[
    (clean_sharks["SightingDateTime"] > '2021-01-01') & (clean_sharks["SightingDateTime"] < '2021-07-01')]

datefiltered_df.head()

,Unnamed: 0,RawDataId,ObjectId,Distance,DistanceUnit,InteractionValue,InteractionId,TownProximity,LocationValue,SightingNumbervalue,ReportDateTime,SightingDateTime,SightingSizeValue,SightingSpeciesValue,OwnerValue,LocationDetail,LocationX,LocationY
36,36,63703,WA53997TSE,30.0,m offshore,sighted,1,Mandurah,"MH055 Fourth Groyne Surf Break BEN Sign, north...",NaN,NaN,2021-06-30T16:10:47,3.0m,unknown sp.,Public report,NaN,115.736,-32.5050
37,37,63702,WA94783TSE,50.0,m offshore,sighted,1,Augusta,"Flinders Bay, Augusta",NaN,NaN,2021-06-30T08:57:04,3.0m,white,Public report,Flinders Bay Beach,115.186,-34.3263
39,39,63699,WA17863TSE,10.0,m offshore,sighted,1,Dunsborough,"Rocky Point Surfing Spot, north of Dunsborough",NaN,2021-06-28T12:00:30,2021-06-28T10:30:54,large,unknown sp.,Public report,NaN,115.059,-33.5457
40,40,63700,WA36733TSE,50.0,m offshore,sighted,1,Bunbury,"Back Beach, Bunbury",NaN,2021-06-28T14:00:05,2021-06-27T14:00:20,NaN,unknown sp.,Public report,NaN,115.629,-33.3272
41,41,63698,WA99425TSE,50.0,m offshore,sighted,1,Denmark,"Parry Beach, west of Denmark",NaN,2021-06-26T11:30:14,2021-06-26T10:00:10,large,white,Public report,NaN,117.160,-35.0381


In [114]:
# save data at this stage for group use

export_data = datefiltered_df[['InteractionValue','InteractionId','SightingSpeciesValue','SightingDateTime','OwnerValue','LocationX','LocationY']]
export_data.to_csv('data/sharks/sharks_cleaned.csv')

Round GPS coordinates for grouping 

In [11]:
datefiltered_df['LocationX'] = round(sharks_df['LocationX'],2)
datefiltered_df['LocationY'] = round(sharks_df['LocationY'],2)
sharks_df

,Unnamed: 0,RawDataId,ObjectId,Distance,DistanceUnit,InteractionValue,InteractionId,TownProximity,LocationValue,SightingNumbervalue,ReportDateTime,SightingDateTime,SightingSizeValue,SightingSpeciesValue,OwnerValue,LocationDetail,LocationX,LocationY
0,0,63739,3073,NaN,NaN,Detected,4,NaN,Meelup,NaN,NaN,2021-07-24T17:48:00,NaN,white,Fisheries Advise,NaN,115.09,-33.57
1,1,63738,3073,NaN,NaN,Detected,4,NaN,Meelup,NaN,NaN,2021-07-24T17:43:00,NaN,white,Fisheries Advise,NaN,115.09,-33.57
2,2,63737,3073,NaN,NaN,Detected,4,NaN,Meelup,NaN,NaN,2021-07-24T17:38:00,NaN,white,Fisheries Advise,NaN,115.09,-33.57
3,3,63736,WA12582TSE,50.0,m offshore,sighted,1,Perth Metro,"Jervoise Bay Beach, Perth Metro",NaN,NaN,2021-07-24T12:15:12,2.5m,unknown sp.,Public report,NaN,115.76,-32.14
4,4,63735,WA51495TSE,600.0,m offshore,sighted,1,Coral Bay,"Bateman Bay, near Coral Bay",NaN,NaN,2021-07-24T11:24:29,NaN,whale carcass,Fisheries advise,Lots of sharks around carcass,113.81,-23.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17309,17309,54674,WA00004TSE,10.0,m,Sighted,1,Dunsborough,"Bunker Bay, near Dunsborough",1,NaN,2014-10-14T03:40:00,2m,white,Public report,NaN,115.04,-33.54
17310,17310,54675,WA00003TSE,NaN,NaN,Sighted,1,Yallingup,"Yallingup Beach, Yallingup",1,NaN,2014-10-14T02:08:00,NaN,unknown sp.,Public report,NaN,115.03,-33.63
17311,17311,54676,WA00002TSE,NaN,NaN,Sighted,1,Dunsborough,"Dunsborough Boat Ramp, Dunsborough",1,2014-10-13T04:00:00,2014-10-13T03:55:00,4.5m,white,Public report,Old Dunsborough,115.11,-33.60
17312,17312,46307,1035,NaN,NaN,Detected,4,NaN,Garden Island (north end),NaN,NaN,2014-10-13T02:28:00,NaN,tiger,Fisheries Advise,NaN,115.64,-32.14


In [37]:
sharks_grouped = sharks_df.copy().groupby(['LocationX','LocationY','SightingSpeciesValue']).agg({
    'SightingSpeciesValue':'first',
    'LocationX':"first",
    'LocationY':'first',
    'SightingDateTime':'count'})

In [38]:
sharks_grouped

SightingSpeciesValue  LocationX  \
LocationX LocationY SightingSpeciesValue                                   
113.06    -25.84    whale carcass               whale carcass     113.06   
113.08    -25.26    white                               white     113.08   
113.15    -26.14    white                               white     113.15   
          -24.75    unknown sp.                   unknown sp.     113.15   
113.53    -25.93    tiger                               tiger     113.53   
...                                                       ...        ...   
123.55    -34.36    white                               white     123.55   
123.56    -34.37    white                               white     123.56   
124.87    -14.44    unknown sp.                   unknown sp.     124.87   
126.05    -32.27    bronze whaler               bronze whaler     126.05   
                    unknown sp.                   unknown sp.     126.05   

                                          LocationY  SightingDateTime  
LocationX LocationY SightingSpeciesValue                               
113.06    -25.84    whale carcass            -25.84                 1  
113.08    -25.26    white                    -25.26                 1  
113.15    -26.14    white                    -26.14                 1  
          -24.75    unknown sp.              -24.75                 1  
113.53    -25.93    tiger                    -25.93                 2  
...                                             ...               ...  
123.55    -34.36    white                    -34.36                 5  
123.56    -34.37    white                    -34.37                17  
124.87    -14.44    unknown sp.              -14.44                 1  
126.05    -32.27    bronze whaler            -32.27                 1  
                    unknown sp.              -32.27                 1  

[2058 rows x 4 columns]

In [50]:
sharks_spotted = sharks_df.copy().groupby(['LocationX','LocationY']).agg({
    'LocationX':"first",
    'LocationY':'first',
    'SightingDateTime':'count'}).rename(columns=
{"SightingDateTime":"Sightings"})

In [51]:
sharks_spotted

LocationX  LocationY  Sightings
LocationX LocationY                                 
113.06    -25.84        113.06     -25.84          1
113.08    -25.26        113.08     -25.26          1
113.15    -26.14        113.15     -26.14          1
          -24.75        113.15     -24.75          1
113.53    -25.93        113.53     -25.93          2
...                        ...        ...        ...
123.47    -16.82        123.47     -16.82          1
123.55    -34.36        123.55     -34.36          5
123.56    -34.37        123.56     -34.37         17
124.87    -14.44        124.87     -14.44          1
126.05    -32.27        126.05     -32.27          2

[1025 rows x 3 columns]

In [52]:
shark_test = sharks_spotted.reset_index(drop=True)
shark_test

,LocationX,LocationY,Sightings
0,113.06,-25.84,1
1,113.08,-25.26,1
2,113.15,-26.14,1
3,113.15,-24.75,1
4,113.53,-25.93,2
...,...,...,...
1020,123.47,-16.82,1
1021,123.55,-34.36,5
1022,123.56,-34.37,17
1023,124.87,-14.44,1


In [54]:
shark_test.to_csv("data/sharks/shark_locations.csv")